In [1]:
from abc import ABC, abstractmethod
from typing import List
import pandas as pd
import docx
import os
import subprocess

from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw
import random
import string
import textwrap

In [2]:
"""
Simple QuoteModel implementation class to encapsulate the body and author
"""
class QuoteModel:
    """
    Implementation class to encapsulate a quote body and a quote author.
    The purpose of our quote generator contains a body and an author
    """

    def __init__(self, body: str, author: str):
        """Create a simple model using a quote author and a quote body."""
        self.author = str(author)
        self.body = str(body)

    def __str__(self):
        """Override method representing the QuoteModel class objects as a string"""
        return f"{self.body} \n Quote by {self.author}"

    def __repr__(self):
        """Override method used to represent a class's objects as a string"""
        return f"Quote Model Author: {self.author}, Quote Model body: {self.body}"


In [3]:
class IngestorInterface(ABC):

    accepted_files = []
    
    @classmethod
    def can_ingest(cls, path: str) -> bool:

        flag = path.split('.')[-1] in cls.accepted_files
        return flag
    
    @classmethod
    @abstractmethod
    def parse(cls, path: str) -> List[QuoteModel]:
        pass

In [4]:
class CSVInterface(IngestorInterface):
    """A class that handles the ingestion of csv files."""

    accepted_files = ["csv"]

    @classmethod
    def parse(cls, path: str) -> List[QuoteModel]:
        """Parse QuoteModel obj of csv files and return a list of them."""
        if not cls.can_ingest(path):
            raise ValueError(f'File {path} cannot be processed')

        
        df = pd.read_csv(path)
        quotes = [QuoteModel(df.loc[idx, 'body'], df.loc[idx, 'author']) for idx in range(len(df))]
        return quotes

In [5]:
csv_quote = CSVInterface()
csv_quote.parse("..\\_data\\DogQuotes\\DogQuotesCSV.csv")

[Quote Model Author: Skittle, Quote Model body: Chase the mailman,
 Quote Model Author: Mr. Paws, Quote Model body: When in doubt, go shoe-shopping]

In [6]:
class TXTInterface(IngestorInterface):
    """A txt ingestion handler class """

    accepted_files = ["txt"]

    @classmethod
    def parse(cls, path: str) -> List[QuoteModel]:
        """
        Parses QuoteModel objects of txt files,
        returning a list of QuoteModels.
        """
        if not cls.can_ingest(path):
            raise ValueError(f'File {path} cannot be processed')

        with open(path, "r", encoding="utf-8-sig") as infile:
            filelines = infile.readlines()
        
        rows = [row.replace("\n", "").split("-") for row in filelines]
        quotes = [QuoteModel(quote[0].strip(), quote[1].strip()) for quote in rows]
        return quotes

In [7]:
txt_quote = TXTInterface()
txt_quote.parse("..\\_data\\DogQuotes\\DogQuotesTXT.txt")

[Quote Model Author: Bork, Quote Model body: To bork or not to bork,
 Quote Model Author: Stinky, Quote Model body: He who smelt it...]

In [8]:
class DOCXInterface(IngestorInterface):
    """A txt ingestion handler class"""

    accepted_files = ["docx"]

    @classmethod
    def parse(cls, path: str) -> List[QuoteModel]:
        """
        Parses QuoteModel objects of docx files,
        returning a list of QuoteModels.
        """
        if not cls.can_ingest(path):
            raise ValueError(f'File {path} cannot be processed')
            
        doc = docx.Document(path)
        
        rows = [row.text.split('-') for row in doc.paragraphs if row.text != ""]
        quotes = [QuoteModel(quote[0].strip(), quote[1].strip()) for quote in rows]


        return quotes

In [9]:
docx_quote = DOCXInterface()
docx_quote.parse("..\\_data\\DogQuotes\\DogQuotesDOCX.docx")

[Quote Model Author: Rex, Quote Model body: "Bark like no one’s listening",
 Quote Model Author: Chewy, Quote Model body: "RAWRGWAWGGR",
 Quote Model Author: Peanut, Quote Model body: "Life is like peanut butter: crunchy",
 Quote Model Author: Tiny, Quote Model body: "Channel your inner husky"]

In [10]:
class PDFInterface(IngestorInterface):
    """A pdf ingestion handler class"""

    accepted_files = ["pdf"]

    @classmethod
    def parse(cls, path: str) -> List[QuoteModel]:
        """
        Parses QuoteModel objects of pdf files,
        returning a list of QuoteModels.
        """
        if not cls.can_ingest(path):
            raise ValueError(f'File {path} cannot be processed')
        # temporary file creation aiming to read and save pdftotext output
        temp_file = f".temporary_file_to_delete.txt"
        
        # calling pdftotext tool
        subprocess.run(["pdftotext", "-layout", path, temp_file])
        # iterate through temporary txt file
        with open(temp_file, "r") as infile:
            lines = infile.readlines()
            rows = [line.strip().split("-") for line in lines]
            quotes = [QuoteModel(quote[0], quote[-1]) for quote in rows if (len(quote[0])>0 | len(quote[-1])>0) ]
        
        os.remove(temp_file) # Remove the created txt file.
        return quotes

In [11]:
pdf_quote = PDFInterface()
pdf_quote.parse("..\\_data\\DogQuotes\\DogQuotesPDF.pdf")

[Quote Model Author:  Fluffles, Quote Model body: "Treat yo self" ,
 Quote Model Author:  Forrest Pup, Quote Model body: "Life is like a box of treats" ,
 Quote Model Author:  Bark Twain, Quote Model body: "It's the size of the fight in the dog" ]

In [12]:
class Ingestor(IngestorInterface):
    """
    File manager encapsulation ingestors deciding which Interface ingestor is appropriate.
    It Returns a list of Quote Models as we have working it ouut in the different Interfaces
    for each file type ingestion .
    """

    ingestorInterfaces = [CSVInterface, DOCXInterface, TXTInterface, PDFInterface]

    @classmethod
    def parse(cls, path: str) -> List[QuoteModel]:
        """
        Iterate our differet ingestor Interfaces 
        and run the approprate can_ingest method 
        It returns the list of Quote Models
        """
        for ingestor in cls.ingestorInterfaces:
            if ingestor.can_ingest(path):
                return ingestor.parse(path)

In [13]:
Ingestor = Ingestor()

In [14]:
Ingestor.parse("..\\_data\\DogQuotes\\DogQuotesCSV.csv")

[Quote Model Author: Skittle, Quote Model body: Chase the mailman,
 Quote Model Author: Mr. Paws, Quote Model body: When in doubt, go shoe-shopping]

In [15]:
Ingestor.parse("..\\_data\\DogQuotes\\DogQuotesTXT.txt")

[Quote Model Author: Bork, Quote Model body: To bork or not to bork,
 Quote Model Author: Stinky, Quote Model body: He who smelt it...]

In [16]:
Ingestor.parse("..\\_data\\DogQuotes\\DogQuotesDOCX.docx")

[Quote Model Author: Rex, Quote Model body: "Bark like no one’s listening",
 Quote Model Author: Chewy, Quote Model body: "RAWRGWAWGGR",
 Quote Model Author: Peanut, Quote Model body: "Life is like peanut butter: crunchy",
 Quote Model Author: Tiny, Quote Model body: "Channel your inner husky"]

In [17]:
Ingestor.parse("..\\_data\\DogQuotes\\DogQuotesPDF.pdf")

[Quote Model Author:  Fluffles, Quote Model body: "Treat yo self" ,
 Quote Model Author:  Forrest Pup, Quote Model body: "Life is like a box of treats" ,
 Quote Model Author:  Bark Twain, Quote Model body: "It's the size of the fight in the dog" ]

In [18]:
class ImageGenerator(object):

    def __init__(self, output_directory):
        self.output_directory = output_directory

    def create_meme(self, image, body, author, width=500) -> str:
        try:
            img = Image.open(image)
            img_aspect_ratio = img.width/img.height
            mm_width, mm_height = 500, int(500*img_aspect_ratio)
            resized_img = img.resize((mm_width, mm_height))

            quote = QuoteModel(body, author).__str__()
            quote_printout = textwrap.fill(quote, 50) if len(quote) > 50 else quote

            x = int(mm_width/16)
            y = int(mm_height/16)

            draw_meme = ImageDraw.Draw(resized_img)
            draw_meme.text((x, y), text=quote_printout, font=ImageFont.truetype("verdana.ttf", 15))
            
            output_path = os.path.join(self.output_directory, 
                                       ''.join(random.choice(string.ascii_letters) for x in range(15)) + '.png')
            
            resized_img.save(output_path)
            return output_path
        
        except Exception as e:
            raise Exception(e)

In [19]:
ImageGenerator = ImageGenerator("..\\_data\\meme_directory")

In [20]:
ImageGenerator.create_meme("..\\_data\\photos\\dog\\xander_1.jpg", "To live or not to live", "William Shakespeare")

'..\\_data\\meme_directory\\jrkZVYsxAjJqjdL.png'

In [21]:
ImageGenerator.create_meme("..\\_data\\photos\\dog\\xander_2.jpg", \
                         "Be yourself; everyone else is already taken", "Oscar Wilde")

'..\\_data\\meme_directory\\fLqINKhLTfjPKAH.png'

In [22]:
ImageGenerator.create_meme("..\\_data\\photos\\dog\\xander_3.jpg",\
                         "I've learned that people will forget what you said,\
people will forget what you did, but people will never forget how you made them feel.", "Maya Angelou")

'..\\_data\\meme_directory\\eTRGsnXIANYPHJU.png'

In [23]:
ImageGenerator.create_meme("..\\_data\\photos\\dog\\xander_4.jpg", "Here's to the crazy ones.\
                        The misfits.\
                        The rebels.\
                        The troublemakers.\
                        The round pegs in the square holes.\
                        The ones who see things differently.\
                        They're not fond of rules. \
                        And they have no respect for the status quo.\
                        You can quote them, disagree with them, glorify or vilify them.\
                        About the only thing you can't do is ignore them.\
                        Because they change things. \
                        They push the human race forward. \
                        And while some may see them as the crazy ones, we see genius.\
                        Because the people who are crazy enough to think they can change the world,\
                        are the ones who do","Rob Siltanen")

'..\\_data\\meme_directory\\QFBVtcGhJRGoeEF.png'